In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from uszipcode import SearchEngine, SimpleZipcode, Zipcode

#read in zip.csv
zip_pd = pd.read_csv("zip.csv")
#clear any null values
zip_pd.dropna(inplace= True)
#filter df to only include standard zip codes(exclude unique and p.o. boxes)
std_pd= zip_pd.loc[zip_pd.Type=="Standard"]

FileNotFoundError: [Errno 2] File b'zip.csv' does not exist: b'zip.csv'

In [ ]:
#filter to only include zip codes in Dallas County
county_zip_pd= std_pd.loc[(std_pd["County"]== "Dallas") | (std_pd["County"]== "Harris")]
#extract all zip codes in Dallas County to a list
county_zips = county_zip_pd["Zip Code"].tolist()
#create lists for lat and long per zip code
lat=[]
long=[]
for z in county_zips:
    search= SearchEngine()
    zipcode= search.by_zipcode(z)
    latitude= zipcode.lat
    lat.append(latitude)
    longitude= zipcode.lng
    long.append(longitude)
#create dictionary
county_zips_dict={"Zip_Codes": county_zips,"Lat":lat, "Lon":long}
#convert list to DF in order to merge later with Census data
county_zip_df= pd.DataFrame(county_zips_dict)
county_zip_df.head()

In [ ]:
apikey= "f342c203f985f826ddbfc242fce4a634199dbcd5"
#from config import api_key
c = Census(apikey,year=2017)
# run api call for general tableid/variables
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zip_Codes"})
#convert Zip Codes to numeric, in order to merge later
census_pd.Zip_Codes= pd.to_numeric(census_pd.Zip_Codes)
census_pd.head()

In [ ]:
# merge Zip DF with Census DF
zip_census_merge= pd.merge(county_zip_df,census_pd, on="Zip_Codes")
zip_census_merge.head(10)

In [ ]:
#census data table info: https://www.census.gov/programs-surveys/acs/guidance/which-data-tool/table-ids-explained.html
# ACS Census Table search https://factfinder.census.gov/faces/nav/jsf/pages/searchresults.xhtml?refresh=t
# variable list https://api.census.gov/data/2017/acs/acs5/variables.html
#census data documentation https://jtleider.github.io/censusdata/
# census table id variables explained https://www.census.gov/programs-surveys/acs/guidance/which-data-tool/table-ids-explained.html

In [ ]:
#### pull population data for 15-44 year olds, the demographic most likely to order delivery####
#from config import api_key
c = Census(apikey,year=2017)
#get male population data for 15-44yr olds
male_age_data = c.acs5.get(("NAME", "B01001_006E","B01001_007E","B01001_008E","B01001_009E","B01001_010E","B01001_011E",
                      "B01001_012E","B01001_013E","B01001_014E"), {'for': 'zip code tabulation area:*'})
#get female population data for 15-44yr olds
female_age_data = c.acs5.get(("NAME", "B01001_030E","B01001_031E","B01001_032E","B01001_033E","B01001_034E","B01001_035E",
                      "B01001_036E","B01001_037E","B01001_038E"), {'for': 'zip code tabulation area:*'})
# Convert to DataFrame
male_pd = pd.DataFrame(male_age_data)
male_pd= male_pd.rename(columns={"zip code tabulation area": "Zip_Codes"})
female_pd= pd.DataFrame(female_age_data)
female_pd= female_pd.rename(columns={"zip code tabulation area": "Zip_Codes"})

# Total population by gender
male_pd["Total_Male"]= male_pd.B01001_006E + male_pd.B01001_007E + male_pd.B01001_008E + male_pd.B01001_009E \
    + male_pd.B01001_010E + male_pd.B01001_011E + male_pd.B01001_012E + male_pd.B01001_013E + male_pd.B01001_014E
female_pd["Total_Female"]= female_pd.B01001_030E + female_pd.B01001_031E + female_pd.B01001_032E + female_pd.B01001_033E \
    + female_pd.B01001_034E + female_pd.B01001_035E + female_pd.B01001_036E + female_pd.B01001_037E + female_pd.B01001_038E
#rename zip code tabulation area to Zip_Codes to align with other dataframes
#male_pd= male_pd.rename(columns={"zip code tabulation area": "Zip_Codes"})
#female_pd= female_pd.rename(columns={"zip code tabulation area": "Zip_Codes"})
#merge male and female df's
gender_merge= pd.merge(male_pd,female_pd, on="Zip_Codes")
#calculate new column for total 15-44 yr olds
gender_merge.loc[:,"Total"]=gender_merge.Total_Male + gender_merge.Total_Female
# filter DF to include only Total and Zip Codes
gender_df = gender_merge[["Total", "Zip_Codes"]]
#convert Zip_codes to numeric
gender_df.Zip_Codes= pd.to_numeric(gender_df.Zip_Codes)

In [ ]:
#merge gender_df with zip_census_merge
demographic_df= pd.merge(zip_census_merge,gender_df, on="Zip_Codes")
demographic_df.head(10)

In [ ]:
gmaps.configure(api_key="")
figure_layout={
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'
}
locations= demographic_df[["Lat","Lon"]]
total = demographic_df["Total"].astype(float)
fig= gmaps.figure(map_type="TERRAIN", layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=total, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig